In [82]:
# We recommend updating pip to the latest version.
!pip install --upgrade pip
# Update matplotlib for plotting charts
!pip install --upgrade matplotlib
# Update Azure Machine Learning SDK to the latest version
!pip install --upgrade azureml-sdk
# For Jupyter notebook widget used in samples
!pip install --upgrade azureml-widgets
# For Tensorboard used in samples
!pip install --upgrade azureml-tensorboard
# Install Azure Machine Learning Reinforcement Learning SDK
!pip install --upgrade azureml-contrib-reinforcementlearning

     |████████████████████████████████| 1.5 MB 3.5 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2
    Uninstalling pip-20.2:
      Successfully uninstalled pip-20.2
Requirement already up-to-date: matplotlib in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (3.3.0)
  Using cached azureml_sdk-1.11.0-py3-none-any.whl (4.4 kB)
  Using cached azureml_pipeline-1.11.0-py3-none-any.whl (3.7 kB)
  Using cached azureml_train_automl_client-1.11.0-py3-none-any.whl (97 kB)
  Using cached azureml_train-1.11.0-py3-none-any.whl (3.2 kB)
  Using cached azureml_core-1.11.0-py3-none-any.whl (2.0 MB)
  Using cached azureml_dataset_runtime-1.11.0-py3-none-any.whl (3.2 kB)
  Using cached azureml_pipeline_core-1.11.0-py3-none-any.whl (295 kB)
  Using cached azureml_pipeline_steps-1.11.0-py3-none-any.whl (58 kB)
  Using cached azureml_automl_core-1.11.0-py3-none-any.whl (152 kB)
  Using cached azureml_telemetry-1.11.0-py3-none-any.whl (29 kB)
  Using cached az

In [104]:
import azureml.core
print('Azure Machine Learning SDK Version: ', azureml.core.VERSION)

Azure Machine Learning SDK Version:  1.9.0


In [105]:
from azureml.core.authentication import InteractiveLoginAuthentication

InteractiveLoginAuthentication(force=False, tenant_id='3e2ec0d3-feb3-40d4-b46e-2a62921527ec', cloud=None)

In [106]:
from azureml.core import Workspace

# ws = Workspace.from_config()

ws = Workspace.get(name="EveML",
               subscription_id='f7aefc60-a918-42a6-91fc-0daf244f3951',
               resource_group='mslearn')

print(ws.name, ws.location, ws.resource_group, sep=' | ')

EveML | northeurope | mslearn


In [107]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='particle-multiagent')

In [108]:
from azureml.core.compute import AmlCompute, ComputeTarget

cpu_cluster_name = 'cpu-cl-d3'

if cpu_cluster_name in ws.compute_targets:
    cpu_cluster = ws.compute_targets[cpu_cluster_name]
    if cpu_cluster and type(cpu_cluster) is AmlCompute:
        if cpu_cluster.provisioning_state == 'Succeeded':
            print('Found existing compute target for {}. Using it.'.format(cpu_cluster_name))
        else: 
            raise Exception('Found existing compute target for {} '.format(cpu_cluster_name)
                            + 'but it is in state {}'.format(cpu_cluster.provisioning_state))
else:
    print('Creating a new compute target for {}...'.format(cpu_cluster_name))
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_D3',
        min_nodes=0, 
        max_nodes=1)

    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_config)
    cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    print('Cluster created.')


Found existing compute target for cpu-cl-d3. Using it.


In [109]:
import os
from azureml.core import Environment
    
cpu_particle_env = Environment(name='particle-cpu')

cpu_particle_env.docker.enabled = True
cpu_particle_env.docker.base_image = 'akdmsft/particle-cpu'
cpu_particle_env.python.interpreter_path = 'xvfb-run -s "-screen 0 640x480x16 -ac +extension GLX +render" python'

max_train_time = os.environ.get('AML_MAX_TRAIN_TIME_SECONDS', 2 * 60 * 60)
cpu_particle_env.environment_variables['AML_MAX_TRAIN_TIME_SECONDS'] = str(max_train_time)
cpu_particle_env.python.user_managed_dependencies = True

In [110]:
from azureml.contrib.train.rl import ReinforcementLearningEstimator
from azureml.widgets import RunDetails

estimator = ReinforcementLearningEstimator(
    source_directory='files',
    entry_script='particle_train.py',
    script_params={
        '--scenario': 'simple_spread',
        '--final-reward': -400
    },
    compute_target=cpu_cluster,
    environment=cpu_particle_env,
    max_run_duration_seconds=3 * 60 * 60
)

train_run = exp.submit(config=estimator)

RunDetails(train_run).show()

_RLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 'sdk_v…

In [111]:
# If you wish to cancel the run before it completes, uncomment and execute:
#train_run.cancel()

In [112]:
import time
from azureml.tensorboard import Tensorboard

head_run = None

timeout = 60
while timeout > 0 and head_run is None:
    timeout -= 1
    
    try:
        head_run = next(r for r in train_run.get_children() if r.id.endswith('head'))
    except StopIteration:
        time.sleep(1)

tb = Tensorboard([head_run])
tb.start()

https://rlcompute-6006.northeurope.instances.azureml.net


'https://rlcompute-6006.northeurope.instances.azureml.net'

In [113]:
from azureml.core import Dataset
from azureml.data.dataset_error_handling import DatasetValidationError

from IPython.display import clear_output
from IPython.core.display import display, Video

datastore = ws.get_default_datastore()
path_prefix = './tmp_videos'

def download_latest_training_video(run, video_checkpoint_counter):
    run_artifacts_path = os.path.join('azureml', run.id)
    
    try:
        run_artifacts_ds = Dataset.File.from_files(datastore.path(os.path.join(run_artifacts_path, '**')))
    except DatasetValidationError as e:
        # This happens at the start of the run when there is no data available
        # in the run's artifacts
        return None, video_checkpoint_counter
    
    video_files = [file for file in run_artifacts_ds.to_path() if file.endswith('.mp4')]
    if len(video_files) == video_checkpoint_counter:
        return None, video_checkpoint_counter
    
    iteration_numbers = [int(vf[vf.rindex('video') + len('video') : vf.index('.mp4')]) for vf in video_files]
    latest_video = next(vf for vf in video_files if vf.endswith('{num}.mp4'.format(num=max(iteration_numbers))))
    latest_video = os.path.join(run_artifacts_path, os.path.normpath(latest_video[1:]))
    
    datastore.download(
        target_path=path_prefix,
        prefix=latest_video.replace('\\', '/'),
        show_progress=False)
    
    return os.path.join(path_prefix, latest_video), len(video_files)


def render_video(vf):
    clear_output(wait=True)
    display(Video(data=vf, embed=True, html_attributes='loop autoplay width=50%'))

In [ ]:
import shutil

terminal_statuses = ['Canceled', 'Completed', 'Failed']
video_checkpoint_counter = 0

while head_run.get_status() not in terminal_statuses:
    video_file, video_checkpoint_counter = download_latest_training_video(head_run, video_checkpoint_counter)
    if video_file is not None:
        render_video(video_file)
        
        print('Displaying video number {}'.format(video_checkpoint_counter))
        shutil.rmtree(path_prefix)
    
    # Interrupting the kernel can take up to 15 seconds
    # depending on when time.sleep started
    time.sleep(15)
    
train_run.wait_for_completion()
print('The training run has reached a terminal status.')

Displaying video number 2


In [73]:
# to stop the Tensorboard, uncomment and run
# tb.stop()

# to delete the cpu compute target, uncomment and run
# cpu_cluster.delete()